In [1]:
import pandas as pd
import nltk
from nltk.stem import SnowballStemmer
from pattern.es import parsetree
import re
from collections import defaultdict
import numpy as np
import unidecode
from textblob import TextBlob
import time
#from spellchecker import SpellChecker
#from googletrans import Translator
#from translate import Translator
from classifier import SentimentClassifier as sf
clf = sf()

#translator = Translator()
#translator = Translator("spanish", "english")

#spell = SpellChecker(language='es', distance=2) # Spanish dictionary
#spell.word_frequency.load_words(['jubilación', 'desahucio'])

palabras_funcionales = nltk.corpus.stopwords.words("spanish")    
stemmer = SnowballStemmer('spanish')

In [2]:
data = pd.read_csv("Data\\preguntas.csv", sep=',', encoding = "ISO-8859-1")
data.head(3)

,Pregunta,respuesta,Tema,Pers/empresa,Polarity,Nota,ambiguedad
0,Buenos días. Mi nombre es Oscar Castro. Ayer ...,NaN,Contacto,NaN,0,NaN,NaN
1,quiero saber si ustedes tratan de temas labor...,NaN,Renuncia/Despido/Desahucio,NaN,NaN,NaN,NaN
2,se puede jubilar con 28 años de aporte y 61 d...,NaN,Jubilacion Patronal,NaN,NaN,NaN,NaN


In [3]:
try:
    data = pd.concat(g for _, g in data.groupby(["Pregunta",'Tema']) if len(g) > 1)
    print(data)
except:
    print('no duplicates')

no duplicates


In [4]:
data.Tema.value_counts()

Jubilacion Patronal             300
IESS                            176
Consultoria                     145
Renuncia/Despido/Desahucio       92
No Topic                         90
Greeting                         62
Contacto                         57
Hi Five                          44
Otros servicios                  43
trabaje con nosotros             43
Charlas/Capacitaciones           35
Facturacion/Retencion/Cobros     25
Queja                            24
Name: Tema, dtype: int64

In [5]:
data['target'] = data.Tema.map({'Jubilacion Patronal': 0, 'Consultoria':1, 'Renuncia/Despido/Desahucio':2, 'IESS':3, 
                                'Greeting':4, 'Contacto':5, 'No Topic':6, 'Queja':7, 'Otros servicios':8, 
                                    'Charlas/Capacitaciones':9, 'Hi Five':10, 'trabaje con nosotros':11, 'Facturacion/Retencion/Cobros':12})

In [6]:
#def spell_check(texto):
#    tokens = nltk.word_tokenize(texto, "spanish")
#    corr_tokens = []
    
#    for token in tokens: 
#        if spell[token]:
#            corr_tokens.append(token)
#        else:
#            corr_tokens.append(spell.correction(token))
       
#    texto2 = " ".join(str(item) for item in corr_tokens)
#    return texto2


def hasNumbers(string):
    return bool(re.search(r'\d', string))


def hasBC(string):
    i = string.find('/')
    return bool(i != -1)


def other_check(token):    
    b1 = not hasNumbers(token)
    b2 = not hasBC(token)
    return (b1 and b2)

def remove_accent(word):
    word = unidecode.unidecode(word)
    word = word.replace('?','')
    word = word.replace('.','')
    return word

def stem_lemma(word):
    stemmer = SnowballStemmer('spanish')
    word = parsetree(word, lemmata=True)[0].lemmata[0]
    word = stemmer.stem(word) 
    return word


def token_and_clean(texto):
    try:          
        tokens = nltk.word_tokenize(texto, "spanish")
        token_list = []
        for token in tokens:        
            if token not in palabras_funcionales:
                token = stem_lemma(token)
                token = remove_accent(token)
                if len(token) >= 2 and other_check(token):
                    token_list.append(token)
                
        if not len(token_list) > 0:
            print('No Tokens in: {0}'.format(texto))
        return token_list
    except:
        print('error en el texto: {0}'.format(texto))        

                
    
def analyze_token(texto, tema, tokens_limpios):
    try:
        token_list = token_and_clean(texto)
        for t in token_list:
            t_list = tokens_limpios[t]
            t_list[0] += 1
            t_list[int(tema + 1)] += 1
            tokens_limpios[t] = t_list
        return tokens_limpios
    except:
        print('error en el texto: {0}'.format(texto))

        
def vectorize_phrase(texto, vocab):
    tokens = token_and_clean(texto)
    vector = np.zeros(len(vocab))
    for t in tokens:
        if t in vocab:
            vector[vocab.index(t)] = 1
    return vector
    
    
def n_token(sentence):
    token_list = token_and_clean(sentence) 
    return len(token_list) 


def polarity(message): #blob has a limit on api calls
    #blob = TextBlob(message)
    
    try:
        #leng = blob.detect_language()
        #leng = 'es'
        
        #if leng != 'en':
            #blob = blob.translate(to='en').lower() 
            #traduccion = translator.translate(message)
            #blob = TextBlob(traduccion.text).lower()
            #blob = TextBlob(traduccion).lower()
        #else:
            #blob = blob.lower() 
            
        pol = clf.predict(message)
    
    except Exception as e:
        print('Exception en polarity: {0}'.format(e))
        pol = 0
        
            
    #pol = blob.sentiment[0]
    
    return pol


def percent_greet(sentence):
    tgreet = ['hol', 'buen', 'tard', 'dia', 'noch']
    count = 0
    tokens = token_and_clean(sentence)
    for w in tokens:       
        if w in tgreet:
            count += 1  
    if len(tokens) > 0:   
        return count/len(tokens) 
    else:
        return 0
    
def ex_capac(sentence):
    tcap = ['charl', 'curs', 'capacit', 'seminari', 'formacion', 'capacitacion']
    count = 0
    tokens = token_and_clean(sentence)
    for w in tokens:       
        if w in tcap:
            count += 1  
    return (count > 0)*1
    
    


In [7]:
tokens_limpios =  defaultdict(lambda: np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])) 

In [8]:
for sent, tema in zip(data.Pregunta, data.target):
    #sent = spell_check(sent)
    tokens_limpios = analyze_token(sent, tema, tokens_limpios)

No Tokens in: cuando


In [9]:
#Calcular las x primeras deben concentrar >= x porcentaje 

In [10]:
for key in tokens_limpios:    
    mlista = tokens_limpios[key]
    if np.argmax(mlista[1:]) != 6 and mlista[0] > 0:  #vocabulary not in no topic
    #if np.argmax(mlista[1:]) != 6:
        mlista = np.array([mlista[0], np.max(mlista[1:]/mlista[0]), np.sum(np.sort(mlista)[-3:-1])/mlista[0]])
    else:
        mlista = np.array([0,0, 0])
    tokens_limpios[key] = mlista  

tokens_limpios

defaultdict(<function __main__.<lambda>()>,
            {'buen': array([303.        ,   0.33333333,   0.51485149]),
             'dia': array([168.        ,   0.27380952,   0.45833333]),
             'mi': array([31.        ,  0.61290323,  0.83870968]),
             'nombr': array([19.        ,  0.26315789,  0.47368421]),
             'oscar': array([1., 1., 1.]),
             'castr': array([1., 1., 1.]),
             'ayer': array([4.  , 0.25, 0.5 ]),
             'llam': array([16.    ,  0.5   ,  0.6875]),
             'empres': array([134.        ,   0.45522388,   0.61940299]),
             'solict': array([1., 1., 1.]),
             'habl': array([0, 0, 0]),
             'sr': array([1., 1., 1.]),
             'rodrig': array([1., 1., 1.]),
             'ibarr': array([1., 1., 1.]),
             'me': array([62.        ,  0.25806452,  0.41935484]),
             'senal': array([3.        , 0.33333333, 0.66666667]),
             'intent': array([1., 1., 1.]),
             'hoy': arr

In [11]:
counts = 3
f_tokens = {k: v for k, v in tokens_limpios.items() if (v[0] >= counts and v[1] > 1/v[0] and v[2] >= 0.4) }
f_tokens

{'buen': array([303.        ,   0.33333333,   0.51485149]),
 'dia': array([168.        ,   0.27380952,   0.45833333]),
 'mi': array([31.        ,  0.61290323,  0.83870968]),
 'nombr': array([19.        ,  0.26315789,  0.47368421]),
 'llam': array([16.    ,  0.5   ,  0.6875]),
 'empres': array([134.        ,   0.45522388,   0.61940299]),
 'me': array([62.        ,  0.25806452,  0.41935484]),
 'hoy': array([8.  , 0.25, 0.5 ]),
 'ser': array([82.        ,  0.35365854,  0.65853659]),
 'posibl': array([12.  ,  0.5 ,  0.75]),
 'contact': array([20.  ,  0.55,  0.75]),
 'quer': array([100.  ,   0.3 ,   0.48]),
 'sab': array([91.        ,  0.27472527,  0.47252747]),
 'si': array([155.        ,   0.36129032,   0.60645161]),
 'usted': array([45.        ,  0.31111111,  0.46666667]),
 'trat': array([5. , 0.4, 0.6]),
 'tem': array([26.        ,  0.30769231,  0.53846154]),
 'laboral': array([35.        ,  0.25714286,  0.51428571]),
 'empleador': array([10. ,  0.5,  0.7]),
 'sin': array([10. ,  0.4,  

In [12]:
vocab = list(f_tokens.keys())
print(len(vocab))
np.save('Data\\vocab.npy', vocab)

607


In [13]:
X = vectorize_phrase(data.Pregunta[0], vocab)
for sent in data.Pregunta[1:]:
    vector = vectorize_phrase(sent, vocab)
    X = np.vstack((X, vector))    

No Tokens in: cuando


In [14]:
X.shape

(1136, 607)

In [15]:
#tot_sum = np.sum(X, axis =1)
#X = np.column_stack((X, tot_sum))

In [16]:
nt = [n_token(x) for x in data.Pregunta]
X = np.column_stack((X, nt))

No Tokens in: cuando


In [17]:
X.shape

(1136, 608)

In [18]:
pg = [percent_greet(x) for x in data.Pregunta]
X = np.column_stack((X, pg))

No Tokens in: cuando


In [19]:
cap = [ex_capac(x) for x in data.Pregunta]
X = np.column_stack((X, cap))

No Tokens in: cuando


In [20]:
X.shape

(1136, 610)

In [21]:
pol = []    

In [22]:
#counter = 0
#for sent in data.Pregunta:
    #time.sleep(0.05)
#    p = polarity(sent)
#    pol.append(p)
#    print('{0} of {1}'.format(counter+1,len(data.Pregunta)))
    
#    counter+=1

pol =  np.load('Data\\pol.npy', allow_pickle=True)

In [23]:
#np.save('Data\\pol.npy', pol)

In [24]:
X = np.column_stack((X, pol))

In [25]:
X.shape

(1136, 611)

In [26]:
np.save('Data\\X.npy', X)

In [27]:
#X2 = np.load('X.npy', allow_pickle=True)


In [28]:
y = data.target.to_numpy()

In [29]:
np.save('Data\\y.npy', y)